In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-28 18:12:23--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.78MB/s    in 0.2s    

2021-06-28 18:12:24 (4.78 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [10]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [11]:

from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [12]:
# filter dataframe for total votes that are equal to or greater than 20.
df1 = df.filter(df.total_votes >= 20)
df1.count()

35312

In [13]:
# create new DF where helpful votes divided by total votes that equal greater than 50%
df2= df1.filter((df1.helpful_votes / df1.total_votes) >= 0.5)
df2.count()

31830

In [23]:
# filter the DF and create new DF showing all rows where a paid review is present as part of the vine program (v=='y')
paid_df = df2.filter(df2.vine == 'Y')
paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21084288|R3QIQRG107AP19|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|           38|         41|   Y|                N|Essential home ow...|We are not pros. ...| 2015-08-27|
|         US|   52512530|R1ZNBHGCYE8VN1|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|    

In [24]:
# filter the DF and create new DF showing all rows where a paid review is not present as part of the vine program (v=='n')
unpaid_df = df2.filter(df2.vine == 'N')
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

In [26]:
# Determine the total number of reviews for Vine containing products.
total_paid = paid_df.count()
total_paid

285

In [31]:
# Determine the total number fo 5 star reviews for Vine containing products.
paid_five = paid_df.filter(paid_df.star_rating == 5).count()
paid_five

163

In [29]:
# Determine the percentage of 5 star reviews for Vine containing products.
paid_percent = (paid_five/total_paid)*100
paid_percent

57.19298245614035

In [30]:
# Determine the total number of reviews for NON Vine containing products.
total_unpaid = unpaid_df.count()
total_unpaid

31545

In [32]:
# Determine the total number fo 5 star reviews for NON Vine containing products.
unpaid_five = unpaid_df.filter(unpaid_df.star_rating ==5).count()
unpaid_five

14614

In [34]:
# Determine the percentage of 5 star reviews for NON Vine containing products.
unpaid_percent = (unpaid_five/total_unpaid)*100
unpaid_percent

46.32746869551435